In [ ]:
from common import *
import geopandas

### Description

Show the difference of different sample perio

In [ ]:
# use_hvplot()

In [ ]:
client = start_cluster(5, 12)
client_ip_and_port(client)

In [ ]:
year = 2021
water_year = f"wy{year}"

wy_snow = xr.open_mfdataset(
    (SNOBAL_DIR / f'GMD/{water_year}' / 'erw/*/snow.nc').as_posix(),
    parallel=True,
)

In [ ]:
days_6 = xr.date_range(wy_snow.time.values[0], periods=55, freq='6D')
days_12 = xr.date_range(wy_snow.time.values[0], periods=25, freq='12D')

diff_daily = wy_snow.specific_mass.diff('time').compute()
diff_6_days = wy_snow.specific_mass.sel(time=days_6).diff('time').compute()
diff_12_days = wy_snow.specific_mass.sel(time=days_12).diff('time').compute()

In [ ]:
melt_daily = abs(diff_daily.where(diff_daily < 0)).fillna(0)
melt_6_days = abs(diff_6_days.where(diff_6_days < 0)).fillna(0)
melt_12_days = abs(diff_12_days.where(diff_12_days < 0)).fillna(0)

client.shutdown()

metl_days_na = melt_days
melt_days = melt_days.fillna(0).where(melt_days > 0)

In [ ]:
sum_daily = melt_daily.sum('time')
sum_6_days = melt_6_days.sum('time')
sum_12_days = melt_12_days.sum('time')

In [ ]:
PLOT_OPTS_HV = dict(
    height=800, width=1080, cmap='bgy_r', cbar_kwargs={"label": "SWE (mm)"}
)

In [ ]:
PLOT_OPTS_MPL = dict(
    cmap='YlGnBu',
    cbar_kwargs={
        "label": "SWE (mm)",
    }
)
plt.rcParams.update(
    {
        'axes.titlesize': 9,
        'axes.labelsize': 6,
        'xtick.labelsize': 6,
        'ytick.labelsize': 6,
    }
)

In [ ]:
ERW = geopandas.read_file(DATA_DIR.joinpath('Boundaries/Gunnison_basin_boundaries.geojson').as_posix())

In [ ]:
def style_mpl(melt_data):
    ERW.loc[ERW['label'] == 'Upper Gunnison'].boundary.plot(ax=ax, color='black')

    ax.set_title('Upper Gunnison - East River Watershed')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelrotation=45)
    ax.ticklabel_format(axis='both', style='plain')
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')

In [ ]:
fig = plt.figure(dpi=300)
ax = fig.gca()

sum_daily.plot(ax=ax, **PLOT_OPTS_MPL)
style_mpl(ax)

In [ ]:
fig = plt.figure(dpi=300)
ax=fig.gca()

sum_6_days.plot(**PLOT_OPTS_MPL)

style_mpl(ax)
ax.set_title('Upper Gunnison - East River Watershed (6 days)')

In [ ]:
fig = plt.figure(dpi=300)
ax=fig.gca()

sum_12_days.plot(**PLOT_OPTS_MPL)

style_mpl(ax)
ax.set_title('Upper Gunnison - East River Watershed (12 days)')

In [ ]:
PLOT_OPTS_MPL['cbar_kwargs']['label'] = ''

In [ ]:
fig = plt.figure(dpi=300)
ax=fig.gca()

(1 - sum_6_days / sum_daily).plot(**PLOT_OPTS_MPL, vmax=0.4, vmin=0.0)

style_mpl(ax)
ax.set_title('')

In [ ]:
fig = plt.figure(dpi=300)
ax=fig.gca()

(1 - sum_12_days / sum_daily).plot(**PLOT_OPTS_MPL, vmax=0.4, vmin=0.0)

style_mpl(ax)
ax.set_title('')

## Basin total

daily_swe.hvplot(label='Daily Total').opts(height=960, width=1280) * \
melt_days.cumsum().hvplot(label='Total SWE diff') *\
abs(melt_days.diff('time')).cumsum().hvplot(label='Daily SWE diff')

In [ ]:
fig = plt.figure(dpi=300)
ax=fig.gca()

(1 - sum_6_days / sum_daily).plot.hist(ax=ax, bins=np.arange(0.0, 1, 0.01), label='6 days', histtype='step',)
(1 - sum_12_days / sum_daily).plot.hist(ax=ax, bins=np.arange(0.0, 1, 0.01), label='12 days', histtype='step',)

plt.xlabel('Percent Error')
plt.ylabel('')
plt.yticks([])
plt.xlim(0,1)

plt.legend()

In [ ]:
(1 - sum_6_days / sum_daily).mean()

In [ ]:
(1 - sum_12_days / sum_daily).mean()